# Исследование надёжности заёмщиков

Датасет содержит различные хараткеристики клиентов банка и данные об их платёжеспособности.
Необходимо определить, влияет ли семейное положение и количество детей и дохрд клиента на факт погашения кредита в срок.


Значения столбцов датасета:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита.

## 1. Обзор данных

Установим pymystem3 для стемминга.

In [1]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Для стемминга.

from pymystem3 import Mystem
from collections import Counter

In [3]:
df = pd.read_csv('datasets/pil.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Таблица содержит 12 колонок и 21 525 строк.
<br>В таблице присутсвуют как целочисленные и вешественные, так и строковые данные.

## 2. Предобработка данных

### 2.1. Обработка пропусков

Проверим значения строк каждого столбца на возможные проблемы.
<br>Начнем с `children`.

In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце children 76 + 47 строк имеют нереалистичное значение. Вероятно, ошибки при ручном заполнении.
<br>Скорректируем эти строк убрав "0" и "-" и проверим корректность этой замены.

In [6]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим значения столбца `days_employed`.

In [7]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Явные проблемы, которые будут решены ниже (2.2  Замена типа данных).

Проверим значения столбца `dob_years`.

In [8]:
print(df['dob_years'].min(), df['dob_years'].max())
df['dob_years'].unique()

0 75


array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

Значения кажутся корректными, кроме клиентов с возрастом 0.
<br>Предположим, что под этими значениями имелся ввиду минимальный возраст заемщика и заменим их на 18 лет.

In [9]:
df['dob_years'] = df['dob_years'].replace(0 , 18)
print(df['dob_years'].min(), df['dob_years'].max())

18 75


Проверим значения столбца `education`.

In [10]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Привежем все строки столбца `education` к нижнему регистру.

In [11]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверяем значения столбца `education_id`.

In [12]:
df['education_id'].value_counts()

# С ним все в порядке - индексация соответсвует столбцу education.

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Проверим значения столбца `family_status`.

In [13]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Ошибок нет, но стоит привести значения к нижнему регситру.

In [14]:
df['family_status'] = df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Проверяем значения столбца `family_status_id`.

In [15]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

С ним все в порядке - индексация соответсвует столбцу `family_status`.

Проверим значения столбца `gender`.

In [16]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Заменим значение `XNA` на `M`, это не изменит статистику, но придаст данным однородность в анализе.
<br>А также приведем все к нижнему регистру.

In [17]:
df['gender'] = df['gender'].replace('XNA', 'M')
df['gender'] = df['gender'].str.lower()
df['gender'].value_counts()

f    14236
m     7289
Name: gender, dtype: int64

Проверяем значения столбца `income_type`.

In [18]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

С ним все в порядке.

Проверим значения столбца `debt`.

In [19]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

С ним все в порядке.

Проверим значения столбца `total_income`.

In [20]:
df['total_income'].value_counts(ascending=False)

112874.418757    1
122421.963500    1
109113.601678    1
164320.213254    1
91053.547017     1
                ..
66950.657779     1
108775.641128    1
225081.030374    1
71919.479823     1
150014.128510    1
Name: total_income, Length: 19351, dtype: int64

С ним все в порядке.

In [21]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Следующие колонки содержат строки с пропущенными значениями: days_employed, total_income.
<br>В контексте данного кейса, эти пропуски принадлежат безработным (например, особенности информационной системы отмечать безработных таким образом).

### 2.2. Замена типа данных

Заполним отсутсвующие данные медианными значениями.

In [22]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())


# Предположим, что отрицательные значения на самом деле должны быть положительными.
# Прогоним по строкам функцию, домножающую на -1 отрицатлеьные значения и округляющую вещественные значения.

def to_positive_number(row):
    if row < 0:
        return row*(-1)
    else:
        return round(row)
    row = row.astype('int')

# Применяем функцию к каждой строке необходимых столбцов.

df['days_employed'] = df['days_employed'].apply(to_positive_number)
df['total_income'] = df['total_income'].apply(to_positive_number)

df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253876,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145886,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267629,дополнительное образование
4,0,340266.000000,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255764,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240526,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135824,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95857,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144426,покупка жилья для семьи


In [23]:
df['days_employed'].describe()

count     21525.000000
mean      60277.957434
std      133301.582080
min          24.141633
25%        1025.608174
50%        1808.053434
75%        4779.587738
max      401755.000000
Name: days_employed, dtype: float64

Значения по-прежнему выглядят неправдоподобно. Найдем максимальное значение `days_employed`, приведенное к годам.

In [24]:
df['days_employed'].max() / 365

1100.6986301369864

1100 лет - неправдоподобное значение, так как максимально возможное число трудовых лет - 50 (трудовой возрас в РФ находится в диапазоне от 16 до 65).
<br>Опрделим, как много значений более 65 лет в датасете.

In [25]:
df.loc[(df['days_employed'] / 365 > 65) == True]['days_employed'].count()

3445

Таких значений 3445. Заменим их на максимальное число лет трудового стажа - 65 и добавим столбец years_employed.

In [26]:
def days_to_years(value):
        if value / 365 > 65:
            return int(65)
        elif value / 365 <= 0:
            return int(1)
        else:
            return int(value / 365)
        
df['years_employed'] = df['days_employed'].apply(days_to_years)

df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253876,покупка жилья,23
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145886,покупка жилья,15
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267629,дополнительное образование,11
4,0,340266.000000,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,65
5,0,926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255764,покупка жилья,2
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240526,операции с жильем,7
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135824,образование,0
8,2,6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95857,на проведение свадьбы,18
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144426,покупка жилья для семьи,5


In [27]:
# Проверим, как теперь распределяется возраст.

df['years_employed'].value_counts(ascending=False)

3     3734
65    3445
1     2023
2     1878
0     1827
4     1350
5     1024
6      965
7      810
8      748
9      572
10     455
11     379
12     345
13     284
14     251
15     205
16     134
19     124
17     117
18      98
20      90
21      88
22      81
23      65
24      53
25      51
26      50
27      46
30      34
29      30
32      28
31      27
28      25
33      17
34      12
35      12
36      10
37       8
38       7
39       6
40       4
41       3
43       3
44       2
42       2
50       1
48       1
45       1
Name: years_employed, dtype: int64

Столбцы `days_employed` и `total_income` имели 10% строк с пропущенными значениями. Чтобы не терять из-за этого существенный объем данных (не удалять строки с отсутсвующими значениями этих столбцов), было принято решение заменить пропущенные значения на медианные по столбцу. Эта "искуственная" подстановка данных приведет к минимальным искжениям общей картины датасета.
После подстановки медианного значения в `days_employed` имеем

### 2.3. Обработка дубликатов

Приведем строки к нижнему регистру.

In [28]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()


df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
 12  years_employed    21525 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


In [29]:
# Проверим, есть ли в датасете явные дубликаты строк.

df.duplicated().sum()

# В датасете присутсвует 71 явный дубликат, избавимся от них.

df = df.drop_duplicates().reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
 12  years_employed    21454 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB


В столбцах со строковыми значенями дубликаты имеет только колонка `purpose`.
Для поиска дубликатов все столбцы со строковыми значениями были приведены к нижнему регистру; далее в каждом столбце была опреелена частотность значений; значения

### 2.4 Лемматизация целей кредитования

Составим классификацию целей кредитования, исходя из неформализованного описания целей кредитования в колонке `purpose`.
Для этого, выделим леммы.

In [30]:
m = Mystem()

text = []
for row in df['purpose']:
    text.append(row)

text = ' '.join(text)

lemmas = m.lemmatize(text)
print(Counter(lemmas))

Installing mystem to C:\Users\Roman/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


Встречается всего 4 цели кредитования:
* недвижимость,
* автомобиль,
* образование,
* свадьба.

Создадим столбец `purpose_category` и добавляем в него значение, соответсвующее найденному в списках вариантов описания цели кредита.

In [31]:
purpose_list = []

purpose = list(df['purpose'].unique())

for value in range(len(purpose)):
    if 'недвиж' in purpose[value]:
        purpose_list.append('операции с недвижимостью')
    elif 'жиль' in purpose[value]:
        purpose_list.append('операции с недвижимостью')
    elif 'авто' in purpose[value]:
        purpose_list.append('операции с автомобилем')
    elif 'образован' in purpose[value]:
        purpose_list.append('получение образования')
    elif 'свадь' in purpose[value]:
        purpose_list.append('проведение свадьбы')
    else:
        purpose_list.append('N/A')
        
        
purpose_dict = dict(zip(purpose, purpose_list))
df['purpose_category'] = df['purpose'].map(purpose_dict)

df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253876,покупка жилья,23,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,11,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145886,покупка жилья,15,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267629,дополнительное образование,11,получение образования
4,0,340266.000000,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,65,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224792,операции с жильем,12,операции с недвижимостью
21450,0,343937.000000,67,среднее,1,женат / замужем,0,f,пенсионер,0,156000,сделка с автомобилем,65,операции с автомобилем
21451,1,2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89673,недвижимость,5,операции с недвижимостью
21452,3,3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093,на покупку своего автомобиля,8,операции с автомобилем


Посмотрим на долю каждлй цели кредитования в числе всех заявок на кредит.

In [32]:
print(round(df['purpose_category'].value_counts() / 21454 * 100), 2)

операции с недвижимостью    50.0
операции с автомобилем      20.0
получение образования       19.0
проведение свадьбы          11.0
Name: purpose_category, dtype: float64 2


Наибольшим спросом пользуются кредиты на недвижимость: 50% клиентов предпочитает брать их; около 20% - занимают деньги банка на покупку автомобиля или образование; 11% - на свадьбу. 

## 3. Анализ возврата кредита различными группами заемщиков

На основе предобработанных данных, попытаемся ответить на следующие вопросы:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для определения зависимости между наличием детей и фактом возврата кредита, найдем долю клиентов, имеющих долг по внутри каждой группы клиентов по числу детей.

In [33]:
children_rate = (df.groupby('children')['debt'].sum() / df.groupby('children')['debt'].count() * 100).round(2)
children_rate.sort_values()

children
5    0.00
0    7.54
3    8.18
1    9.17
2    9.49
4    9.76
Name: debt, dtype: float64

Таким образом, наиболее надежными заемщиками являются клиенты с 5 детьми, однако в данной подгруппе всего 9 клиентов, что нерепрезентативно.
Клиенты без детей, вероятнее, не будут иметь задолжность по воврату кредитов, однако, существенной завимости наличия задолженности от числа детей, не выявлено.

Для определения, есть ли зависимость между семейным положением и возвратом кредита в срок, найдем долю клиентов, имеющих долг по кредиту внутри каждой группы клиентов по семейному статусу.

In [34]:
family_status_rate = (df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count() * 100).round(2)
family_status_rate.sort_values()

family_status
вдовец / вдова           6.57
в разводе                7.11
женат / замужем          7.55
гражданский брак         9.35
не женат / не замужем    9.75
Name: debt, dtype: float64

Из этих данных очевидно, вдовцы имеют наибольшую долю просроченых кредитов. Люди, не имеющие официальных отношений - наименьшую.
Вероятно, они более дисциплинированы и лучше контролируют свой бюджет.

Для определения зависимости между уровнем дохода и возвратом кредита в срок, сгруппируем доход по квантилям и рассплитим клиентов на 4 группы, добавив новый столбец в датасет.

In [35]:
income_groups = df['total_income'].quantile([0.1, 0.5, 0.9])
income_groups

0.1     78721.8
0.5    145018.0
0.9    269798.1
Name: total_income, dtype: float64

In [36]:
# Функция для рассплитовки клиентов по доходу.

def income_groupping(total_income):
    if total_income < 78812.2:
        return 'Низкий'
    elif total_income < 145018.0:
        return 'Средний'
    elif total_income < 269535.4:
        return 'Высокий'
    else:
        return 'Очень высокий'

df['total_income_group'] = df['total_income'].apply(income_groupping)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,purpose_category,total_income_group
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253876,покупка жилья,23,операции с недвижимостью,Высокий
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,11,операции с автомобилем,Средний
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145886,покупка жилья,15,операции с недвижимостью,Высокий
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267629,дополнительное образование,11,получение образования,Высокий
4,0,340266.000000,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,65,проведение свадьбы,Высокий


In [37]:
# Распределение числа клиентов по данным группам.

df['total_income'].mean()
df.groupby('total_income_group')['debt'].count()

total_income_group
Высокий          9626
Низкий           2153
Очень высокий    2153
Средний          7522
Name: debt, dtype: int64

In [38]:
# Определим долю клиентов, имеющих долг по кредиту внутри каждой группы клиентов по уровню дохода.

total_income_rate = (df.groupby('total_income_group')['debt'].sum() / df.groupby('total_income_group')['debt'].count() * 100).round(2)
total_income_rate.sort_values()

total_income_group
Очень высокий    7.01
Низкий           7.34
Высокий          8.17
Средний          8.59
Name: debt, dtype: float64

Таким образом, наименьшая задолженность наблюдается среди клиентов с уровнем дохода выше 0,9 процентиля (очень высокий доход).
Наибольшая задолженность - среди клиентов с уровнем дохоов выше 0,1 процентиля, но ниже медианы (средний доход).

Если для людей с очень высоким доходом, низкая доля задолжности по возврату очевидна, то для клиентов с низким уровнем дохода
она может объясняться небольшими суммами кредита (соответсвенно, относительной легкостью их выплаты), однако в датасете нет тикетов кредита, чтобы проверить эту гипотезу.
Аналогично: высокая доля задолженности среди клиентов среднего и высокого дохода также объясняется более высокими суммами тикета.

Для определения того, как разные цели кредита влияют на его возврат в срок, найдем долю клиентов, имеющих долг по кредиту внутри каждой группы клиентов по сгруппированным целям кредита.

In [39]:
purpose_rate = (df.groupby('purpose_category')['debt'].sum() / df.groupby('purpose_category')['debt'].count() * 100).round(2)
purpose_rate.sort_values(ascending=False)

purpose_category
операции с автомобилем      9.36
получение образования       9.22
проведение свадьбы          8.00
операции с недвижимостью    7.23
Name: debt, dtype: float64

Наибольшую задолженность по кредиту имеют клиенты, бравшие его для покупки автомобиля и получения образования, наименьшую - клиенты, заимствовавшие деньги для операций с недвижимостью. Это может объясняться большей величиной тела крдита и более серьезными обязательствами заемщика (покупка не является спонтанной), а также более жестким скорингом клиента на этапе подготовки предложения.

## 4. Итоги исследования

Зависимость возврата кредита существует со следующими параметрами:
1. Семейным положением: вдовцы имеют наибольшую долю просроченых кредитов. Люди, не имеющие официальных отношений - наименьшую.
2. Уровненем дохода: наименьшая задолженность - среди клиентов с очень высоким уровнем дохода выше 0,9 процентиля; наибольшая  - среди клиентов со средним (выше 0,1 процентиля, ниже медианы).
3. Целью кредита: заимствования на приобретение недвижимости имеют наименьший риск просро

Зависимости между количеством детей у заемщика и рисками кредитования не выявлено, тем не менее, исходя из данных, клиенты без детей, вероятнее, не будут иметь задолжности по воврату кредитов.

Стоит обратить внимание на улучшение информационной системы сбора данных о заемщиках для устранения проблемы с форматом данных (трудовой стаж в днях изменить на месяцы, например и ввести автоматическую проверку данных); ограничить уровень образование и цели кредита набором из выпадающего списка; добавить информацию у сумме и частоте просрочек платежей, чтобы дальнейшие исследовали были более точными (например, техническая и минимальная задержка искажают картину платежеспособности групп клиентов).